In [3]:
import pandas as pd 
import numpy as np
from numpy import array
import os
from pathlib import Path
from collections import namedtuple
import re

### Traversing the folder 

- check the json data
- check the whether the time start and time end are distinct
- check whether there are 5 paragraphs

In [4]:
def extract_user_number(s):
    """
    Extracts the user number from a given path
    :s: path of the file
    """
    s = str(s).split("_")[0] # extract the user from the path
    s = s.split("/")[6] # split the path to isolate the user
    s = s.replace("User","") # remove the word "User"
    s = int(re.sub(r'[a-z]+', '', s, re.I)) # remove alphabetical characters
    return s 


In [5]:
p = Path("/cs/home/ybk1/Dissertation/Experiment Anonymised Version")
# p = Path("C://Users//User//OneDrive - University of St Andrews//Modules//CS5099//4. Data Documents//dataset//INSTRUMENTED DIGITAL AND PAPER READINGDATASET//Experiment Anonymised Version")

In [6]:
def get_all_datasets(p):
    """
    Loops through all the folders to find all the datasets and json files for training
    p: path of the root folder
    """
    File = namedtuple('File', 'name path size')
    files = []
    for item in p.glob('**/*'): # loops thorough all the files in all the sub-directories
        if item.match('*rawEEGData.csv')  and "baseline" not in item.name:
            name = item.name
            path = Path.resolve(item).parent
            size = item.stat().st_size

            files.append(File(name,path, size )) # stores the name, path and size in named tuple
    
    df = pd.DataFrame(files)
    df['user'] = df["path"].apply(extract_user_number)
    df.to_csv("All EEG files.csv", index=False)
    
    return df

From the value counts we can see that: 
- user 20, 23, 17, 7, 9, 15, 2 - all have duplicates
- user 14 is one short


### Building the datasets using the alldatasets csv

### Loading the data 

In [13]:
# s_dir = "/cs/home/ybk1/Dissertation/Experiment Anonymised Version/User001_Group_Test_30-08-2019--10-07-00/User001_test1_BET_01_30-08-2019/User001_test1_BET_01_30-08-2019_recording0/"
# s_csv = "User001_test1_BET_01_30-08-2019_recording0_U1567156556_EEG_rawEEGData.csv"
# s_json = "annotations.json"

In [14]:
# s_data = pd.read_csv(s_dir + s_csv)

In [32]:
# pd_labels = pd.read_json(s_dir + s_json)

In [24]:
def check_json(labels):
    """
    Method for checking the json to see how many paragraphs it contains
    and whether the timestamps are distinct
    :labels: annotations.json in a DataFrame
    """
    #check whether there are 5 paragraphs
    no_para = len(labels)    
    if len(labels) != 5:
        print ("There are less than 5 Paragraraphs in the dataset")
    else:
        print("There are 5 paragraphs in the dataset")
    
    #check whether the timestamps are distincts
    are_timestamps_distinct = False
    cols = ["timeRangeStart", "timeRangeEnd"]
    for col in cols:
        array_length= len(array(labels[col]))
        set_length = len(set(array(labels[col])))
        if array_length == set_length:
            print(col + " has distinct values")
            are_timestamps_distinct = True
            break
        else: 
            print(col + " does not have distinct values")
            are_timestamps_distinct = False
            break
    
    print(are_timestamps_distinct)
    return no_para, are_timestamps_distinct 
    

In [8]:
def label(timestamp, annotation, labels):
    """
    Method for adding the labels to the dataframe
    
    :return: relevant score of attention/interest/effort within the correct time range
    :timestamp: timestamp at the current iteration
    :annotation: annotation at the current interation
    :labels: annotations.json in array format
    """
    for row in labels:
        time_start = row[-2]
        time_end = row[-1]
        ann_dict = {"effort": row[-3], "attention": row[-4], "interest": row[-5],"para": row[-6]}
        if timestamp >= time_start and timestamp < time_end: # checks if the timestamp is witin the start and end range
            return ann_dict[annotation] # returns the relevant score stored in the dictionary
        

In [9]:
def add_labels(data, labels):
    """
    Method for processing the dataset by creating columns for effort, attention, interest and paragraph
    :data: EEG dataset
    :labels: annotations.json in array format
    """
    annotations = ["effort", "attention", "interest", "para"]
    for ann in annotations: # loops through the labels, and creates new columns based on the values within the timestamp range
        data[ann] = data["Timestamp"].apply(label, annotation=ann, labels=labels)
        
    print("Processed: {}".format(len(data)))
    print("Empty rows \n{}".format(data.isnull().sum()))
    data = data.dropna()
    return data   

In [10]:
def export_datasets_to_csv(labeled_data, path):
    """
    Exports the labeled data as three separate csv files for interest, effort and attention
    :labeled_data: annotated datasets with columns for interest, effort and attention
    :path: path to save the dataset
    """
    X = labeled_data.iloc[:,:9] # inputs i.e channels and timestamp
    labels = ['interest', 'effort', 'attention']
    for label in labels:
        dataset = pd.concat([X, labeled_data[label]], axis=1).to_csv(path +  "/" +'EEG_' + label + '_dataset.csv')   


In [11]:
def build_all_datasets(df):
    """
    Method for building all the training sets by applying annotations at relevant timestamps
    and splitting the training sets into attention, interest and effort
    :df: DataFrame that includes all of the datasets
    :ds: dataset
    """
    ds = array(df)
    json = "annotations.json" # json file that contains the labels and annotations for the data
    File = namedtuple('File', 'path user no_para are_timestamps_distinct')
    files = []
    
    for row in ds:
        ds_path = row[1] # Get the path of the dataset
        ds_csv = row[0] # get the name of the csv file
        ds_user = row[3] # get the user number
        ds_data = pd.read_csv(ds_path + "/" + ds_csv)
        ds_labels = pd.read_json(ds_path + "/" + json)
        no_para, are_timestamps_distinct = check_json(ds_labels) # check the json file
        files.append(File(ds_path, ds_user, no_para,are_timestamps_distinct))
        
        
        ds_labels = array(ds_labels)
        if are_timestamps_distinct == False:
            print("Don't process")
        else: 
            labeled_data = add_labels(ds_data, ds_labels) # Gets the labeled data
            export_datasets_to_csv(labeled_data, ds_path) # Exports the 3 datasets to the appropriate path
            
    training_files = pd.DataFrame(files)
        

In [25]:
def build_all_datasets_test():
    """
    Test method for building a single group of training sets by applying annotations at relevant timestamps
    and splitting the training sets into attention, interest and effort
    :df: DataFrame that includes all of the datasets
    :ds: dataset
    """
    
    ds_path = "/cs/home/ybk1/Dissertation/Experiment Anonymised Version/User001_Group_Test_30-08-2019--10-07-00/User001_test0_IET_00_30-08-2019/User001_test0_IET_00_30-08-2019_recording0"
    ds_csv = "User001_test0_IET_00_30-08-2019_recording0_U1567156289_EEG_rawEEGData.csv"
    json = "annotations.json"
    
    File = namedtuple('File', 'path user no_para are_timestamps_distinct')
    files = []
        
    ds_user = 1 # get the user number
    ds_data = pd.read_csv(ds_path + "/" + ds_csv)
    ds_labels = pd.read_json(ds_path + "/" + json)
    no_para, are_timestamps_distinct = check_json(ds_labels) # check the json file
    files.append(File(ds_path, ds_user, no_para,are_timestamps_distinct))


    ds_labels = array(ds_labels)
    if are_timestamps_distinct == False:
        print("Don't process")
    else: 
        labeled_data = add_labels(ds_data, ds_labels) # Gets the labeled data
        export_datasets_to_csv(labeled_data, ds_path) # Exports the 3 datasets to the appropriate path
            
    training_files = pd.DataFrame(files)
    return training_files
        

In [26]:
build_all_datasets_test()

There are 5 paragraphs in the dataset
timeRangeStart does not have distinct values
False
Don't process


,path,user,no_para,are_timestamps_distinct
0,/cs/home/ybk1/Dissertation/Experiment Anonymis...,1,5,False
